In [ ]:
from __future__ import print_function
import argparse
import torch
#import foolbox
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms, utils
from sklearn.decomposition import PCA
import pickle
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA
import argparse
from numpy import ma
import scipy
import os.path
from sklearn.linear_model import LogisticRegression
import pandas as pd
from skimage.measure import compare_ssim as ssim
from skimage import feature
from skimage import color
from skimage import io

from network.cifar10_cnn_net import cifar10_cnn_net
from network.cifar10_ann import ann_net
from network.adv_cifar10_cnn_net import adv_cifar10_cnn_net
from network.adv_cifar10_ann import adv_ann_net
import utils.cw_final as cw_final
import utils.cw as cw

def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



def train_binary(args, model, device, train_loader, optimizer, epoch, target_class):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        batch_size = target.numpy().shape[0]
        
        ## canny edge detection stub
        #print(data.size())
        edge_data = torch.zeros(batch_size, 1024)
        for i in range(batch_size):
            img = color.rgb2gray(data[i].permute(1,2,0).numpy())
            edge_data[i] = torch.Tensor(feature.canny(img, sigma=1.8).astype(float)).reshape(1024)
        
        #data = data.reshape(-1, 784) # Remove this line while training cnn
        edge_data, target = edge_data.to(device), target.to(device)
        
        #print(size)
        for i in range(batch_size):
            if target[i] != target_class:
                #print(target[i])
                target[i] = 1
                #print(target[i])
            else:
                target[i] = 0
        optimizer.zero_grad()
        output = model(edge_data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))

    
def test_binary(args, model, device, test_loader, target_class):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            batch_size = data.size(0)
            #canny edge detection stub
            edge_data = torch.zeros(batch_size, 1024)
            for i in range(batch_size):
                img = color.rgb2gray(data[i].permute(1,2,0).numpy())
                edge_data[i] = torch.Tensor(feature.canny(img, sigma=1.8).astype(float)).reshape(1024)
                if target[i].data != target_class:
                    target[i] = torch.Tensor([1]).type(torch.LongTensor)#.to(torch.LongTensor)
                else:
                    target[i] = torch.Tensor([0]).type(torch.LongTensor)
            
            #data = data.reshape(data.size(0), 784) # Remove this line while training cnn
                
            edge_data, target = edge_data.to(device), target.to(device)
            output = model(edge_data)
            #print(output)
            #test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1)#, keepdim=True)  # get the index of the max log-probability            
            #print(pred[0])
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



In [ ]:
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=100, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=50, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=True,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
parser.add_argument('--save-model', action='store_true', default=True,
                    help='For Saving the current Model')
args = parser.parse_args([])
use_cuda = not args.no_cuda and torch.cuda.is_available() 

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('data/cifar10', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0,), (1,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('data/cifar10', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0,), (1,))
    ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


In [ ]:
model = cifar10_cnn_net().to(device)
if (args.save_model):
        optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
        for epoch in range(1, args.epochs + 1):
            train(args, model, device, train_loader, optimizer, epoch)
            test(args, model, device, test_loader)

        torch.save(model.state_dict(), "cifar_cnn.pt")


Final Model Accuracy:

Test set: Average loss: 0.5847, Accuracy: 8163/10000 (82%)


In [ ]:
# train binary classifiers

model = ann_net().to(device)
optimizer = None
success = 0
for m in range(10):
    # for training of the cnn-network
    if (args.save_model):
        optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
        for epoch in range(1, args.epochs + 1):
            train_binary(args, model, device, train_loader, optimizer, epoch, m)
            test_binary(args, model, device, test_loader, m)

        torch.save(model.state_dict(), "models/cifar_ann_canny/cifar_ann_canny_binary_sigma_1.8_"+ str(m) + ".pt")

Accruacy for binary Classifier 0 
Accruacy for binary Classifier 1 
Accruacy for binary Classifier 2 
Accruacy for binary Classifier 3 
Accruacy for binary Classifier 4 
Accruacy for binary Classifier 5 
Accruacy for binary Classifier 6 
Accruacy for binary Classifier 7 
Accruacy for binary Classifier 8 
Accruacy for binary Classifier 9 

In [ ]:
# UD case: 
#       1) create adv imgs for trained model without defence for different confidence values
#       2) Mix with equal number of natural imgs
#       3) Calculate accuracy with defence
conf_list = [0, 10, 20]
for conf in conf_list:
    model = adv_cifar10_cnn_net()#.to(device)
    model.load_state_dict(torch.load("cifar_cnn.pt"))
    model.eval()

    classifier = cifar10_cnn_net()
    classifier.load_state_dict(torch.load("cifar_cnn.pt"))
    classifier.eval()

    d0 = ann_net()
    d1 = ann_net()
    d2 = ann_net()
    d3 = ann_net()
    d4 = ann_net()
    d5 = ann_net()
    d6 = ann_net()
    d7 = ann_net()
    d8 = ann_net()
    d9 = ann_net()
    detectors = [d0, d1,d2,d3,d4,d5,d6,d7,d8,d9]#.to(device)

    for i in range(10):
        detectors[i].load_state_dict(torch.load("models/cifar_ann_canny/mnist_ann_canny_binary_sigma_1.8_"+\
                                                str(i) + ".pt"))
        detectors[i].eval()

    inputs_box = (0.0, 1.0)
    adversary = cw.L2Adversary(targeted=False,
                                   confidence=conf,
                                   search_steps=9,
                                   box=inputs_box,
                                   optimizer_lr=0.01)
    adv_imgs = []
    counter = 0
    
    for data, target in test_loader:
        adv, l2 = adversary(model, data, target, to_numpy=False)
        for i in range(args.test_batch_size):
            if l2[i] != np.inf:    #check if attack was successful
                adv_imgs.append(adv[i].reshape(32, 32).numpy())
        if counter == 10:
            break
        else:
            counter += 1
            
    with open("adv_imgs/cifar/UD_cifar_adv_imgs_cw_conf_"+str(conf)+".pkl", "wb") as fp:   #Pickling
        pickle.dump(adv_imgs, fp)            
    
    print("UD Case")
    print("Confidence: ", conf)
    print("Attack Success Rate: ", len(adv_imgs) / 1000)
    print("Avg l2: ", np.mean(l2))
    
    true_pos = 0
    false_pos = 0
    true_neg = 0
    false_neg = 0

    for i in range(len(adv_imgs)):
            adv_im = adv_imgs[i]

            # canny edge sdetection stub
            #print(adv_im.shape)
            #sigma = 1.8 gives best results

            output = classifier(torch.Tensor(adv_im).reshape(1, 3, 32, 32))
            #print(output)
            pred = output.argmax(dim=1, keepdim=True)
            #print(pred)
            adv_im = color.rgb2gray(data[i].permute(1,2,0).numpy())
            adv_edge = feature.canny(adv_im, sigma=1.8).astype(float)
            det_out = detectors[pred](torch.Tensor(adv_edge).reshape(1, 1024))
            prediction = det_out.argmax(dim=1, keepdim=True)
            if prediction == 0:
                false_pos += 1
            else:
                true_pos += 1
        #print(m)
        #print(success1/len(train_adv_imgs[m]))
    counter = 0
    for data, target in test_loader:

        data, target = data.to(device), target.to(device)
        output = classifier(data)
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        for i in range(args.test_batch_size):
            prediction = np.asscalar(pred[i].numpy())
            #detector.load_state_dict(torch.load("models/mnist_ann_canny/mnist_ann_canny_binary"+ str(pred) + ".pt"))
            #detector.eval()
            if (prediction != np.asscalar(target[i].numpy())): #ignoring natural errors
                continue 

            img = color.rgb2gray(data[i].permute(1,2,0).numpy())
            edge_data = feature.canny(img, sigma=1.8).astype(float)
            output = detectors[prediction](torch.Tensor(edge_data).reshape(1, 1024))
            prediction = output.argmax(dim=1, keepdim=True)
            if prediction == 0:
                true_neg += 1
            else:
                false_neg += 1
        if counter == 10:
            break
        else:
            counter += 1
    print("True Positive: ", true_pos)
    print("False Positive: ", false_pos)
    print("True Negative: ", true_neg)
    print("False Negative: ", false_neg)

In [ ]:
# KD Case: Targeted
conf_list = [0, 10, 20]
for conf in conf_list:
    model = cnn_net()#.to(device)
    model.load_state_dict(torch.load("cifar_cnn.pt"))
    model.eval()

    inputs_box = (0.0, 1.0)
    adversary = cw_final.L2Adversary(targeted=True,
                                   confidence=conf,
                                   search_steps=20,
                                   box=inputs_box,
                                   optimizer_lr=0.005)

    adv_imgs = [[] for i in range(10)]
    l2_norms = [[] for i in range(10)]
    success = 0
    class_counter = 0    

    for target_class in range(10):

        detector = adv_ann_net()#.to(device)
        detector.load_state_dict(torch.load("models/cifar_ann_canny/cifar_ann_canny_binary_sigma_1.8_"+\
                                            str(target_class)+".pt"))
        detector.eval()

        # re define test-loader with batch_size 100
        test_loader = torch.utils.data.DataLoader(
                    datasets.CIFAR10('data/cifar10', train=False, transform=transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0,), (1,))])),
                    batch_size=100, shuffle=True, **kwargs)
        
        counter = 0
        for data, label in test_loader:
            #data = torch.Tensor(feature.canny(data.reshape(28, 28).numpy(), sigma=1.0).astype(float)).reshape(1,1,28,28)
            #print(label)
            #if np.asscalar(target.numpy()) == 0:
            #   continue
            target = torch.Tensor([target_class]*100).type(torch.LongTensor)#.to(torch.LongTensor)
            adv, batch_l2_norm = adversary(model, detector, data, target, to_numpy=False)


            #assert isinstance(adversarial_examples, torch.FloatTensor)
            #assert adv.size() == inputs.size()
            #print(success)
            for i in range(100):
                #print(adv.size())
                if batch_l2_norm[i] != np.inf and label[i] != target_class:
                    #output = model(adv[i].reshape(1,1,28,28))
                    #pred = output.argmax(dim=1, keepdim=True)
                    #print("out ", pred)
                    #if np.asscalar(pred.numpy()) == np.asscalar(target.numpy()[i]):
                    #    print("appended")
                    success += 1
                    adv_imgs[target_class].append(adv[i])
                    l2_norms[target_class].append(batch_l2_norm[i])
                if label[i] == target_class:
                    class_counter += 1
            if counter == 11:
                break
            else:
                counter += 1
            
    print("KD Case, conf")
    print("success rate: ", success/(1100 - class_counter))
    print("Examples processed: ", 1100 - class_counter)
    l = []
    for i in range(10):
        l = l + l2_norms[i]
    print("mean l2 norm:", np.mean(l))

        '''with open("adv_imgs/cifar/cw_cifar_adv_imgs_conf_"+str(conf)+"_test_loader_KD_targeted_sigma_1.8_"+\
                      str(target_class)+".pkl", "wb") as fp:   #Pickling
            pickle.dump(adv_imgs, fp)
        with open("adv_imgs/cifar/cw_cifar_l2_norms_conf_"+str(conf)+"_test_loader_KD_targeted_sigma_1.8_"+\
                      str(target_class)+".pkl", "wb") as fp:   #Pickling
            pickle.dump(adv_imgs, fp) '''

In [ ]:
# KD Case: Untargeted

conf_list = [0, 10, 20]
for conf in conf_list:
    model = cnn_net()#.to(device)
    model.load_state_dict(torch.load("cifar_cnn.pt"))
    model.eval()

    d0 = ann_net()
    d1 = ann_net()
    d2 = ann_net()
    d3 = ann_net()
    d4 = ann_net()
    d5 = ann_net()
    d6 = ann_net()
    d7 = ann_net()
    d8 = ann_net()
    d9 = ann_net()
    detectors = [d0, d1,d2,d3,d4,d5,d6,d7,d8,d9]#.to(device)

    for i in range(10):
        detectors[i].load_state_dict(torch.load("models/cifar_ann_canny/cifar_ann_canny_binary_sigma_1.8_"+ str(i) + ".pt"))
        detectors[i].eval()

    inputs_box = (0.0, 1.0)
    adversary = cw_final.L2Adversary(targeted=False,
                                     confidence=conf,
                                     search_steps=9,
                                     box=inputs_box,
                                     optimizer_lr=0.01)

    adv_imgs = []
    l2_norms = []
    success = 0
    counter = 0
    zero_counter = 0

    for data, target in test_loader:
        #data = torch.Tensor(feature.canny(data.reshape(28, 28).numpy(), sigma=1.0).astype(float)).reshape(1,1,28,28)
        #print(label)
        #if np.asscalar(target.numpy()) == 0:
        #   continue
        #print(counter)
        #target = torch.Tensor([0]*batch_size).type(torch.LongTensor)#.to(torch.LongTensor)
        adv, batch_l2_norm = adversary(model, detectors, data, target, to_numpy=False)


        #assert isinstance(adversarial_examples, torch.FloatTensor)
        #assert adv.size() == inputs.size()
        #print(success)
        for i in range(batch_size):
            #print(adv.size())
            if batch_l2_norm[i] != np.inf:
                #output = model(adv[i].reshape(1,1,28,28))
                #pred = output.argmax(dim=1, keepdim=True)
                #print("out ", pred)
                #if np.asscalar(pred.numpy()) == np.asscalar(target.numpy()[i]):
                #    print("appended")
                success += 1
                adv_imgs.append(adv[i])
                l2_norms.append(batch_l2_norm[i])
        if counter == 10:
            break
        else
            counter += 1
    print("KD Untargeted: conf: ", conf)    
    print("Success Rate: ", success/1000)
    print("Avg l2 Norm: ", np.mean(l2_norms))
    
    with open("adv_imgs/cifar/cw_KD_untargeted_cifar_adv_imgs_conf_"+str(conf)+\
              "_test_loader_canny_sigma_1.8.pkl", "wb") as fp:   #Pickling
        pickle.dump(adv_imgs, fp)
    with open("adv_imgs/cifar/cw_KD_untargeted_cifar_l2_norms_conf_"+\
              str(conf)+"_test_loader_canny_sigma_1.8.pkl", "wb") as fp:   #Pickling
        pickle.dump(adv_imgs, fp)        
